In [1]:
import polars as pl
import os
print(os.getcwd())

c:\Users\eliot.nehme\repositories\memoire-ia\codes


In [6]:
# On définit explicitement que DEPT est du texte
table_souscription = pl.read_csv("Donnes_souscription.csv")

print(table_souscription)

ComputeError: could not parse `"2B"` as dtype `i64` at column 'DEPT' (column number 6)

The current offset in the file is 16420 bytes.

You might want to try:
- increasing `infer_schema_length` (e.g. `infer_schema_length=10000`),
- specifying correct dtype with the `schema_overrides` argument
- setting `ignore_errors` to `True`,
- adding `"2B"` to the `null_values` list.

Original error: ```invalid primitive value found during CSV parsing```

In [17]:
# avant de décider du type de chaque colonne. 
# C'est un peu plus lent (quelques secondes), mais c'est 100% sûr.

table_souscription = pl.read_csv(
    "../data/Donnees_souscription.csv", 
    infer_schema_length=None  # <--- Le secret est là !
)

print(f"Dimensions : {table_souscription.shape}") # Affiche le nombre de lignes/colonnes
print(table_souscription.head())

Dimensions : (108653, 27)
shape: (5, 27)
┌─────┬─────────────────┬───────────────────┬─────────────────┬───┬─────┬─────┬─────┬─────┐
│     ┆ num_police      ┆ date_debut_police ┆ date_fin_police ┆ … ┆ 2   ┆ 3   ┆ 4   ┆ 5   │
│ --- ┆ ---             ┆ ---               ┆ ---             ┆   ┆ --- ┆ --- ┆ --- ┆ --- │
│ i64 ┆ i64             ┆ str               ┆ str             ┆   ┆ f64 ┆ f64 ┆ f64 ┆ i64 │
╞═════╪═════════════════╪═══════════════════╪═════════════════╪═══╪═════╪═════╪═════╪═════╡
│ 2   ┆ 201000000000002 ┆ 2010-08-26        ┆ 2011-08-26      ┆ … ┆ 0.0 ┆ 0.0 ┆ 0.0 ┆ 0   │
│ 8   ┆ 201000000000008 ┆ 2010-03-22        ┆ 2011-03-22      ┆ … ┆ 0.0 ┆ 0.0 ┆ 0.0 ┆ 0   │
│ 54  ┆ 201000000000054 ┆ 2010-07-06        ┆ 2011-07-06      ┆ … ┆ 0.0 ┆ 0.0 ┆ 0.0 ┆ 0   │
│ 56  ┆ 201000000000056 ┆ 2010-05-13        ┆ 2011-05-13      ┆ … ┆ 0.0 ┆ 0.0 ┆ 0.0 ┆ 0   │
│ 71  ┆ 201000000000071 ┆ 2010-02-24        ┆ 2011-02-24      ┆ … ┆ 0.0 ┆ 0.0 ┆ 0.0 ┆ 0   │
└─────┴─────────────────┴──────────────

In [27]:
import numpy as np
import pandas as pd

In [34]:
df = pd.read_csv('../data/Donnees_souscription.csv')
print(df.head())

   Unnamed: 0       num_police date_debut_police date_fin_police  REGION DEPT  \
0           2  201000000000002        2010-08-26      2011-08-26      52   72   
1           8  201000000000008        2010-03-22      2011-03-22      11   77   
2          54  201000000000054        2010-07-06      2011-07-06      52   85   
3          56  201000000000056        2010-05-13      2011-05-13      76   66   
4          71  201000000000071        2010-02-24      2011-02-24      75   79   

  COMMUNE     CRITAIR                       ENERGIE  AGE_VOIT  ... TRANS GARL  \
0   72341  Crit'air 3                       Essence        10  ...     5    1   
1   77235  Crit'air 2                        Gazole         5  ...     6    1   
2   85226  Crit'air 1  Essence hybride rechargeable         0  ...     5    2   
3   66190  Crit'air 3                       Essence        16  ...     Z    2   
4   79249  Crit'air 2                        Gazole        16  ...     5    1   

   N_COND  ANCIENNETE  N_S

In [39]:
import os

In [ ]:
import pandas as pd
import numpy as np
from datetime import timedelta
import random
import os

# Chargement de la base de données
# J'utilise le chemin vers ton fichier CSV
df = pd.read_csv('../data/Donnees_souscription.csv')

# --- ÉTAPE 1 : Filtrer et Exploser la base ---

# On ne garde que les lignes ayant au moins 1 sinistre
# Note : la colonne dans le CSV est "N_SINISTRE" (singulier)
table_sinistres = df[df['N_SINISTRE'] > 0].copy()

# On "explose" la table : on duplique les lignes selon la valeur de N_SINISTRE
# Si N_SINISTRE = 2, la ligne est répétée 2 fois
table_sinistres = table_sinistres.loc[table_sinistres.index.repeat(table_sinistres['N_SINISTRE'])].reset_index(drop=True)

print(f"Nombre de lignes après explosion (doit correspondre au nombre total de sinistres) : {len(table_sinistres)}")

# --- ÉTAPE 2 : Simulation Actuarielle ---

# Conversion des colonnes dates en format datetime si ce n'est pas déjà fait
table_sinistres['date_debut_police'] = pd.to_datetime(table_sinistres['date_debut_police'])
table_sinistres['date_fin_police'] = pd.to_datetime(table_sinistres['date_fin_police'])

# 1. Simulation du COUT_SINISTRE (Loi Log-Normale)
# Paramètres de la loi normale sous-jacente (mu et sigma)
# Ces valeurs (mu=7, sigma=1.5) sont des exemples standards pour obtenir des montants réalistes.
# Tu peux les ajuster selon la sévérité moyenne désirée.
mu = 7.0
sigma = 1.5

# Génération des coûts aléatoires
table_sinistres['COUT_SINISTRE'] = np.random.lognormal(mean=mu, sigma=sigma, size=len(table_sinistres))

# Arrondi à 2 décimales pour faire "monétaire"
table_sinistres['COUT_SINISTRE'] = table_sinistres['COUT_SINISTRE'].round(2)

# 2. Simulation de la DATE_SINISTRE (Aléatoire entre date_debut et date_fin)
# Calcul de la durée de couverture en jours pour chaque ligne
duree_couverture = (table_sinistres['date_fin_police'] - table_sinistres['date_debut_police']).dt.days

# Génération d'un nombre de jours aléatoire entre 0 et la durée de couverture
jours_aleatoires = np.random.randint(0, duree_couverture + 1, size=len(table_sinistres))

# Ajout de ces jours à la date de début pour obtenir la date du sinistre
table_sinistres['DATE_SINISTRE'] = table_sinistres['date_debut_police'] + pd.to_timedelta(jours_aleatoires, unit='D')

# --- Affichage et Vérification ---
print("\n--- Aperçu de la nouvelle table table_sinistres ---")
print(table_sinistres[['num_police', 'N_SINISTRE', 'date_debut_police', 'DATE_SINISTRE', 'COUT_SINISTRE']].head(100))

# Sauvegarde optionnelle
# table_sinistres.to_csv('data/table_sinistres_generee.csv', index=False)





# --- ÉTAPE 3 : Injection de la Fraude et Association des Images ---

# 1. Définition des chemins vers tes images (Ceux de ton projet TER)
# J'ai repris les chemins de ton PDF, vérifie qu'ils sont toujours bons sur ton PC !
path_true_images = r"C:\Users\jules\OneDrive\Documents\ISFA 2A\2eme semestre\TER\Data\CrashedCar\True data"
path_fake_images = r"C:\Users\jules\OneDrive\Documents\ISFA 2A\2eme semestre\TER\Data\CrashedCar\Fake data"

# Petite sécurité : on vérifie si les dossiers existent, sinon on met un message
if not os.path.exists(path_true_images):
    print(f"ATTENTION : Le dossier {path_true_images} est introuvable. Vérifie le chemin.")
    images_reelles_dispo = []
else:
    # On liste tous les fichiers images (jpg, png) dans le dossier
    images_reelles_dispo = [f for f in os.listdir(path_true_images) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

if not os.path.exists(path_fake_images):
    print(f"ATTENTION : Le dossier {path_fake_images} est introuvable.")
    images_fake_dispo = []
else:
    images_fake_dispo = [f for f in os.listdir(path_fake_images) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]


# 2. Simulation de la Fraude
# On décide arbitrairement qu'il y a 5% de fraudeurs dans les sinistres
# 0 = Honnête, 1 = Fraudeur
table_sinistres['IS_FRAUD'] = np.random.choice([0, 1], size=len(table_sinistres), p=[0.95, 0.05])


# 3. La fonction magique qui associe l'image
def get_image_path(is_fraud):
    # Si le dossier est vide ou introuvable, on met "Image_Manquante"
    if is_fraud == 1:
        if not images_fake_dispo: return "Image_Manquante"
        # On pioche une image au hasard dans le dossier "Fake"
        nom_image = random.choice(images_fake_dispo)
        # On retourne le chemin complet
        return os.path.join(path_fake_images, nom_image)
    else:
        if not images_reelles_dispo: return "Image_Manquante"
        # On pioche une image au hasard dans le dossier "True"
        nom_image = random.choice(images_reelles_dispo)
        return os.path.join(path_true_images, nom_image)

# On applique cette fonction ligne par ligne pour créer la colonne lien
# C'est ici que la fusion opère !
table_sinistres['CHEMIN_IMAGE'] = table_sinistres['IS_FRAUD'].apply(get_image_path)

# --- Résultat Final ---
print("\n--- Aperçu Final avec Images ---")
# On regarde les colonnes intéressantes
print(table_sinistres[['num_police', 'COUT_SINISTRE', 'IS_FRAUD', 'CHEMIN_IMAGE']].head())

# On sauvegarde ce fichier "Hybride" pour la suite de ton mémoire
table_sinistres.to_csv('base_sinistres_hybride.csv', index=False)
print("\nBase sauvegardée sous 'base_sinistres_hybride.csv'. C'est elle que ton IA va lire !")


Nombre de lignes après explosion (doit correspondre au nombre total de sinistres) : 12349

--- Aperçu de la nouvelle table table_sinistres ---
         num_police  N_SINISTRE date_debut_police DATE_SINISTRE  COUT_SINISTRE
0   201000000000077           1        2010-09-26    2010-11-07         375.78
1   201000000000287           1        2010-03-12    2010-12-31        6406.20
2   201000000000321           1        2010-11-05    2011-06-30         103.98
3   201000000000395           1        2010-10-28    2011-08-21        1159.46
4   201000000000419           1        2010-01-17    2010-01-24        1078.37
..              ...         ...               ...           ...            ...
95  201000000012791           1        2010-01-06    2010-02-16         675.11
96  201000000012835           1        2010-04-03    2011-01-30         519.35
97  201000000012945           1        2010-07-11    2011-02-02        1199.19
98  201000000013028           1        2010-11-16    2011-08-27    